In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/Version_3.2.1.txt
/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/readme.txt
/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/train.csv
/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/test.csv


In [2]:
!pip install torch transformers scikit-learn -q

In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup

In [4]:
train=pd.read_csv('/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/train.csv')
train=train.sample(2000)
train.head()

,class_index,review_text
76987,4,I moved away from Pittsburgh 4 years ago and s...
9573,4,I have enjoyed my stay very much. The complime...
111883,3,"Philly's is your basic, no-frills sports bar. ..."
335511,3,Great old fashioned food except for the OWNER'...
57982,3,Decided to try Sonoma Cellar as I had a $25.00...


In [5]:
test=pd.read_csv('/kaggle/input/yelp-reviews-for-sa-finegrained-5-classes-csv/yelp_review_fine-grained_5_classes_csv/test.csv')
test=test.sample(1000)
test.head()

,class_index,review_text
2326,2,I sort of kind of really really wanted to like...
7147,2,As a follow up to my previous review - the pra...
24449,1,"Am glad my hubby liked his sushi, as I had one..."
13649,4,The jerk chicken fried rice was incredible and...
19559,1,Here is my gripe. The food is good. No complai...


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word not in set(stopwords.words('english'))]
    doc = nlp(" ".join(words))
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

train['review_text'] = train['review_text'].apply(preprocess_text)
print(train.head())

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
        class_index                                        review_text
76987             4  move away pittsburgh year ago still miss place...
9573              4  enjoy stay much complimentary breakfast also d...
111883            3  philly basic frill sport bar nothing swoon lay...
335511            3  great old fashioned food except owner audacity...
57982             3  decide try sonoma cellar discount coupon price...


In [7]:
def label_sentiment(class_index):
    if class_index >= 4.0:
        return 2  # positive
    elif class_index <= 2.0:
        return 0  # negative
    else:
        return 1  # neutral

train['sentiment'] = train['class_index'].apply(label_sentiment)
test['sentiment'] = test['class_index'].apply(label_sentiment)


In [8]:
train.isnull().sum()

class_index    0
review_text    0
sentiment      0
dtype: int64

In [9]:
test.isnull().sum()

class_index    0
review_text    0
sentiment      0
dtype: int64

In [10]:
train.head()

,class_index,review_text,sentiment
76987,4,move away pittsburgh year ago still miss place...,2
9573,4,enjoy stay much complimentary breakfast also d...,2
111883,3,philly basic frill sport bar nothing swoon lay...,1
335511,3,great old fashioned food except owner audacity...,1
57982,3,decide try sonoma cellar discount coupon price...,1


In [11]:
test.head()

,class_index,review_text,sentiment
2326,2,I sort of kind of really really wanted to like...,0
7147,2,As a follow up to my previous review - the pra...,0
24449,1,"Am glad my hubby liked his sushi, as I had one...",0
13649,4,The jerk chicken fried rice was incredible and...,2
19559,1,Here is my gripe. The food is good. No complai...,0


In [12]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False,
)

tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def tokenize_pad_truncate(texts, max_len=500):
    encoding = tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    return encoding['input_ids'], encoding['attention_mask']

In [14]:
X_train = train['review_text'].values
y_train = train['sentiment'].values
X_test = test['review_text'].values
y_test = test['sentiment'].values

In [15]:
# Process each split
X_train_ids, X_train_masks = tokenize_pad_truncate(X_train)
X_test_ids, X_test_masks = tokenize_pad_truncate(X_test)

In [16]:
# Convert labels to tensors
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [17]:
# Create TensorDataset
train_data = TensorDataset(X_train_ids, X_train_masks, y_train)
test_data = TensorDataset(X_test_ids, X_test_masks, y_test)

In [18]:
# Create DataLoader
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=8)
test_dataloader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=8)

In [19]:
# load model and initiate optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 2e-05
    maximize: False
    weight_decay: 0.01
)

In [20]:
# Training function
def train(model, train_dataloader, optimizer):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    
    avg_loss = total_loss / len(train_dataloader)
    return avg_loss

In [21]:
# Validation function
def evaluate(model, val_dataloader):
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in val_dataloader:
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits
        total_eval_loss += loss.item()
        preds = torch.argmax(logits, dim=1).flatten()
        total_eval_accuracy += (preds == b_labels).cpu().numpy().mean()
    
    avg_accuracy = total_eval_accuracy / len(val_dataloader)
    avg_loss = total_eval_loss / len(val_dataloader)
    return avg_loss, avg_accuracy

In [23]:
import tensorflow as tf

print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

2024-07-07 14:42:35.504311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 14:42:35.504413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 14:42:35.601958: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0
: /physical_device:GPU:1


In [ ]:
with tf.device('/GPU:0'):
    #training loop
    epochs=4
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        train_loss = train(model, train_dataloader, optimizer)
        val_loss, val_accuracy = evaluate(model, test_dataloader)
        print(f'Train Loss: {train_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

Epoch 1/4


In [ ]:
# Save the model
torch.save(model.state_dict(), 'model.pth')